In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import os

In [3]:
original_data_dir = r"D:\MED_LEAF_ID\data\cnn\original"
augmented_data_dir =  r"D:\MED_LEAF_ID\data\cnn\augmented"


In [4]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet mean & std
])


In [5]:
val_test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
# Loading datasets
train_data = datasets.ImageFolder(root=original_data_dir, transform=train_transforms)
augmented_data = datasets.ImageFolder(root=augmented_data_dir, transform=train_transforms)

# Merging datasets
full_train_data = torch.utils.data.ConcatDataset([train_data, augmented_data])

# Split dataset into train, validation, and test sets
train_size = int(0.7 * len(full_train_data))
val_size = int(0.15 * len(full_train_data))
test_size = len(full_train_data) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_train_data, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [7]:
class EfficientNetB0Classifier(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetB0Classifier, self).__init__()
        self.base_model = torchvision.models.efficientnet_b0(pretrained=True)
        self.base_model.classifier[1] = nn.Linear(self.base_model.classifier[1].in_features, num_classes)
        self.base_model.classifier.add_module('dropout', nn.Dropout(0.2))  # Added dropout

    def forward(self, x):
        return self.base_model(x)

In [9]:
num_classes = len(train_data.classes)
model = EfficientNetB0Classifier(num_classes=num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(device)
criterion = nn.CrossEntropyLoss()

y_train = [label for _, label in train_dataset]
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

d:\MED_LEAF_ID\base\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\MED_LEAF_ID\base\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cuda


In [12]:
def train(model, train_loader, val_loader, optimizer, criterion, num_epochs=30, patience=5):
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_wts = model.state_dict()
    
    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels.data)
            total_preds += labels.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_preds.double() / total_preds

        # Validation
        model.eval()
        val_loss = 0.0
        correct_preds = 0
        total_preds = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                correct_preds += torch.sum(preds == labels.data)
                total_preds += labels.size(0)

        val_loss /= len(val_loader.dataset)
        val_acc = correct_preds.double() / total_preds

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print("Early stopping triggered")
            break

        # Save losses and accuracies for plotting
        train_losses.append(epoch_loss)
        val_losses.append(val_loss)
        train_accuracies.append(epoch_acc)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs} => Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_losses, val_losses, train_accuracies, val_accuracies


In [ ]:
model, train_losses, val_losses, train_accuracies, val_accuracies = train(
    model, train_loader, val_loader, optimizer, criterion, num_epochs=30, patience=5
)


Training Epoch 1/30: 100%|██████████| 302/302 [02:33<00:00,  1.96it/s]


Epoch 1/30 => Train Loss: 4.3637, Train Accuracy: 0.0232, Val Loss: 4.3147, Val Accuracy: 0.0193


Training Epoch 2/30: 100%|██████████| 302/302 [02:37<00:00,  1.91it/s]


Epoch 2/30 => Train Loss: 4.2713, Train Accuracy: 0.0340, Val Loss: 4.2287, Val Accuracy: 0.0372


Training Epoch 3/30:  11%|█         | 32/302 [00:16<02:11,  2.05it/s]

In [12]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:

model.eval()
correct_preds = 0
total_preds = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels.data)
        total_preds += labels.size(0)

test_accuracy = correct_preds.double() / total_preds
print(f"Test Accuracy: {test_accuracy:.4f}")


torch.save(model.state_dict(), "efficientnetb0_leaf_model.pth")


epochs = range(1, len(train_losses) + 1)
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Train Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Train Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


Epoch 1/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 206s 483ms/step - accuracy: 0.2823 - loss: 3.2409 - val_accuracy: 0.1394 - val_loss: 3.8737
Epoch 2/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 236s 577ms/step - accuracy: 0.6513 - loss: 1.4725 - val_accuracy: 0.3333 - val_loss: 2.5155
Epoch 3/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 252s 617ms/step - accuracy: 0.7412 - loss: 1.0896 - val_accuracy: 0.4249 - val_loss: 2.0485
Epoch 4/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 272s 664ms/step - accuracy: 0.7895 - loss: 0.8717 - val_accuracy: 0.4838 - val_loss: 1.8021
Epoch 5/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 261s 638ms/step - accuracy: 0.8146 - loss: 0.7531 - val_accuracy: 0.5550 - val_loss: 1.5519
Epoch 6/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 267s 653ms/step - accuracy: 0.8355 - loss: 0.6679 - val_accuracy: 0.5851 - val_loss: 1.4762
Epoch 7/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 263s 644ms/step - accuracy: 0.8481 - loss: 0.6084 - val_accuracy: 0.6110 - val_loss: 1.3624
Epoch 8/50
409/409 ━━━━━━━━━━━━━━━━━━━━ 259s 633ms/step - accuracy: 0.8565 -

In [ ]:
test_labels = []
test_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        test_labels.extend(labels.cpu().numpy())
        test_preds.extend(preds.cpu().numpy())

print(classification_report(test_labels, test_preds, target_names=train_data.classes))
print("Confusion Matrix:\n", confusion_matrix(test_labels, test_preds))


87/87 ━━━━━━━━━━━━━━━━━━━━ 39s 403ms/step - accuracy: 0.7649 - loss: 0.7847

Test Loss: 0.7587890028953552
Test Accuracy: 0.7768535017967224
